In [1]:
import pickle
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import imageio

import shutil
import os
if not os.getcwd().endswith('haawron_mmaction2'):
    os.chdir('../..')
assert os.getcwd().endswith('haawron_mmaction2')

np.random.seed(999)  # numpy & sklearn

In [2]:
from mmaction.core import (mean_average_precision, mean_class_accuracy,
                    mmit_mean_average_precision, top_k_accuracy,
                    confusion_matrix)

In [3]:
anns = {}
for split_name, p_ann in zip(
    ['train_source', 'train_target', 'valid', 'test'],
    [
        Path(r'data/epic-kitchens-100/filelist_P02_train_open.txt'),  # to extract only closed
        Path(r'data/epic-kitchens-100/filelist_P04_train_open.txt'),
        Path(r'data/epic-kitchens-100/filelist_P04_valid_open.txt'),
        Path(r'data/epic-kitchens-100/filelist_P04_test_open.txt')
    ]
):
    with p_ann.open('r') as f:
        reader = csv.reader(f, delimiter=' ')
        ann = [int(label) for _, _, _, label in reader]
        ann = np.array(ann)
        anns[split_name] = ann
        print(split_name, ann.shape)

train_source (4666,)
train_target (5275,)
valid (1507,)
test (753,)


In [4]:
Xs = {}
# p_root = Path(r'work_dirs/sanity/ek100/tsm/GCD4DA/P02_P04/19135__GCD4DA_tsm_ek100_sanity_one_way/0/20220506-045539')
p_root = Path(r'work_dirs/sanity/ek100/tsm/GCD4DA/P02_P04/19390__GCD4DA_tsm_ek100_sanity_one_way_phase1/0/20220509-222653')
for split_name, p_feature in zip(
    ['train_source', 'train_target', 'valid', 'test'],
    [
        p_root / 'features/P02_train_open.pkl',
        p_root / 'features/P04_train_open.pkl',
        p_root / 'features/P04_valid_open.pkl',
        p_root / 'features/P04_test_open.pkl',
    ]
):
    with p_feature.open('rb') as f:
        X = pickle.load(f)
        X = np.array(X)
        if split_name == 'train_source':
            X = X[anns[split_name]!=5]  # only closed
        Xs[split_name] = X
    print(split_name, X.shape)

train_source (2816, 512)
train_target (5275, 512)
valid (1507, 512)
test (753, 512)


In [5]:

def cosine(X, y, norm_X=None):
    norm_X = norm_X if norm_X is not None else np.einsum('ij,ji->i', X, X.T)  # (X @ X.T).diagnoal()
    sim = X @ y / (norm_X * (y@y))**.5
    sim[(sim>1.)  & (abs(sim-1.)<1e-4)] = 1.
    sim[(sim<-1.) & (abs(sim+1.)<1e-4)] = -1.
    return sim

def get_kmeans_model(k):
    # 1. get initial centroids
    # 1-1. semi. k-means: centroids are mean vector for each class for labeled data
    centroids = [] 
    centroid_args = []
    for c in range(5):
        ann_train_source_closed = anns['train_source'][anns['train_source'] != 5]
        centroids.append(Xs['train_source'][ann_train_source_closed==c].mean(axis=0))
    # 1-2. k-means++: centroids are initialized by random choices w.p. their distances to the last picked centroid
    new_centroid = centroids[-1]
    X = np.concatenate((Xs['train_source'], Xs['train_target']), axis=0)
    norm_X = np.einsum('ij,ji->i', X, X.T)  # (X @ X.T).diagnoal()
    for _ in range(k-5):
        y = new_centroid
        sim = cosine(X, y, norm_X)
        assertion = ((-1 <= sim) & (sim <= 1))
        assert assertion.all(), sim[~assertion]
        d = np.arccos(sim)
        d **= .3  # Heuristics: d = sqrt(theta)
        d[centroid_args] = 0
        centroid_arg = np.random.choice(X.shape[0], p=d/d.sum())
        centroid_args.append(centroid_arg)
        new_centroid = X[centroid_arg]
        centroids.append(new_centroid) 
    centroids = np.array(centroids)
    # 2. train
    model = KMeans(n_clusters=k, init=centroids, n_init=1, tol=1e-7).fit(X)
    return model

def get_model_score(model, verbose=True, os=True):
    scores = {}
    for split_name in ['train_target', 'valid', 'test']:
        if verbose:
            print(split_name.split('_')[0])

        pred_test = model.predict(Xs[split_name])
        pred_test = np.minimum(5, pred_test, dtype=np.int64)
        conf = confusion_matrix(pred_test, anns[split_name])
        
        recalls = conf.diagonal() / conf.sum(axis=1)
        score = 100 * (recalls.mean() if os else recalls[:5].mean())
        scores[split_name] = score
        if verbose:
            print(conf)
            print(f'{score:.1f}')
            print()
    return scores

In [6]:
n_tries = 20
rows_os, rows_os_star = [], []
ks = list(range(5, 16)) + [30, 50, 100]
global_best_model = None
global_best_test_score = 0
global_best_k = 0
for k in ks:
    local_best_model = None
    local_best_scores = None
    local_best_test_score = 0
    for _ in range(n_tries):
        model = get_kmeans_model(k)
        scores_os = get_model_score(model, False)
        scores_os_star = get_model_score(model, False, False)
        current_test_score = scores_os['test']
        if local_best_test_score < current_test_score:
            local_best_model = model
            local_best_scores = scores_os, scores_os_star
            local_best_test_score = current_test_score
    rows_os.append(pd.Series(local_best_scores)[0])
    rows_os_star.append(pd.Series(local_best_scores)[1])
    if global_best_test_score < local_best_test_score:
        global_best_model = local_best_model
        global_best_test_score = local_best_test_score
        global_best_k = k
print(f'best test score {global_best_test_score:.1f} at k={global_best_k}')


/data/hyogun/anaconda3/envs/open-mmlab/lib/python3.7/site-packages/threadpoolctl.py:762: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


best test score 39.0 at k=8


In [9]:
C = global_best_model.cluster_centers_
(C ** 2).mean(axis=1) ** .5

array([0.04222293, 0.04312205, 0.04722004, 0.04772336, 0.05112299,
       0.05018607, 0.05731261, 0.04872426], dtype=float32)

In [17]:
pd.set_option('precision', 1)
df = pd.DataFrame.from_records(rows_os, columns=['train_target', 'valid', 'test'], index=ks)
df.name = 'OS'
df.T

,5,6,7,8,9,10,11,12,13,14,15,30,50,100
train_target,12.3,18.4,20.4,14.5,14.6,14.4,16.1,13.9,19.5,25.2,12.3,16.5,16.3,17.6
valid,12.7,16.0,14.4,13.7,13.7,20.5,16.9,14.1,12.7,14.6,13.6,15.0,20.4,16.5
test,28.2,29.6,28.0,27.9,22.9,26.0,27.8,22.4,21.5,25.7,26.1,19.4,20.5,17.7


In [18]:
df = pd.DataFrame.from_records(rows_os_star, columns=['train_target', 'valid', 'test'], index=ks)
df.name = 'OS*'
df.T

,5,6,7,8,9,10,11,12,13,14,15,30,50,100
train_target,25.1,28.1,10.4,12.8,10.3,6.2,5.3,6.0,16.6,6.8,6.7,9.7,2.7,0.9
valid,17.4,10.8,26.9,6.9,11.1,8.7,9.2,8.8,5.0,5.1,3.0,4.0,1.4,0.8
test,16.9,8.9,12.9,15.7,9.5,18.1,14.0,3.4,13.6,4.0,9.8,3.1,0.5,0.6


In [ ]:
score_os_star = global_best_model.transform(Xs['test'])[:,:5]
pred_os_star = score_os_star.argmin(axis=1)
conf = confusion_matrix(pred_os_star, anns['test'])[:5, :5]
recalls = conf.diagonal() / conf.sum(axis=1)
print(recalls)
print(recalls.mean())

In [ ]:
pred = global_best_model.predict(Xs['test'])
conf = confusion_matrix(np.minimum(pred, 5, dtype=np.int64), anns['test'])
recalls = conf.diagonal() / conf.sum(axis=1)
print(conf)
with np.printoptions(precision=2):
    print(recalls)
    print(f'OS:  {100*recalls.mean():.1f}')

In [ ]:
idx_to_label = ['take', 'put', 'wash', 'open', 'close', 'unknown']
num_samples = 5
correct = np.minimum(pred, 5)==anns['test']
args_correct, args_incorrect = {}, {}
for c in range(global_best_model.n_clusters):
    args = ((pred==c) & correct).nonzero()[0]
    args_correct[c]   = args if args.shape[0] < num_samples else np.random.choice(args, num_samples, replace=False)
    args = ((pred==c) & ~correct).nonzero()[0]
    args_incorrect[c] = args if args.shape[0] < num_samples else np.random.choice(args, num_samples, replace=False)
args_correct, args_incorrect

In [ ]:
from collections import Counter
Counter(pred), Counter(anns['test'])

In [ ]:
p_ann = Path(r'data/epic-kitchens-100/filelist_P04_test_open.txt')  # to extract only closed
with p_ann.open('r') as f:
    reader = csv.reader(f, delimiter=' ')
    clips = [(Path(rel_p), int(start), int(length), int(label)) for rel_p, start, length, label in reader]

In [ ]:
p_base_video_dir = Path(r'data/epic-kitchens-100/EPIC-KITCHENS')
p_out_dir_base = Path('work_dirs/hello/clusters/P02_P04')
max_gif_length = 200

if p_out_dir_base.is_dir():
    shutil.rmtree(p_out_dir_base)

for correctness, args in zip(['o', 'x'], [args_correct, args_incorrect]):
    p_out_dir = p_out_dir_base / correctness
    if not p_out_dir.is_dir():
        p_out_dir.mkdir(parents=True, exist_ok=True)
    for c in range(global_best_model.n_clusters):
        print(c, end='\t')
        for i, (rel_p, start, length, label) in enumerate(np.array(clips)[args[c]]):
            p_video = p_base_video_dir / rel_p
            with imageio.get_writer(p_out_dir / f'k{c:02d}_{i:02d}_{idx_to_label[min(5, c)]}_{idx_to_label[min(5, label)]}.gif', mode='I', duration=1/30) as writer:
                for frame_idx in range(start, start+min(max_gif_length,length)+1):
                    p_frame = p_video / f'frame_{frame_idx:010d}.jpg'
                    image = imageio.imread(p_frame)
                    writer.append_data(image)
            print(i, end=' ')
        print()

In [ ]:
import pickle

p_centroid = p_root / 'centroids.pkl'
with p_centroid.open('wb') as f:
    pickle.dump(global_best_model.cluster_centers_, f)

In [ ]:
.shape

In [ ]:
p_ann_train_target = Path(r'data/epic-kitchens-100/filelist_P04_train_open.txt')  # to extract only closed
with p_ann_train_target.open('r') as f:
    reader = csv.reader(f, delimiter=' ')
    clips = list(reader)

pseudo_labels = global_best_model.predict(Xs['train_target'])
p_pseudo = p_root / f'filelist_pseudo_P04_k{global_best_k:03d}_one_way.txt'
with p_pseudo.open('w') as f:
    writer = csv.writer(f, delimiter=' ')
    for (rel_p, start, length, label), pseudo_label in zip(clips, pseudo_labels):
        writer.writerow([rel_p, start, length, pseudo_label])

In [ ]:
p_pseudo